<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0614/02_4_RNN__%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%83%9D%EC%84%B1(%EC%88%98%EC%97%85%ED%95%84%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 40.3 MB/s eta 0:00:00


In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Bidirectional, Dropout

In [ ]:
text = '''그대 기억이 지난 사랑이

내 안을 파고드는 가시가 되어

제발 가라고 아주 가라고

애써도 나를 괴롭히는데

아픈 만큼 너를 잊게 된다면

차라리 앓고 나면 그만인데

가시처럼 깊게 박힌 기억은

아파도 아픈 줄 모르고

그대 기억이 지난 사랑이

내 안을 파고드는 가시가 되어

제발 가라고 아주 가라고'''

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

32

In [ ]:
print(tokenizer.word_index)

{'가라고': 1, '그대': 2, '기억이': 3, '지난': 4, '사랑이': 5, '내': 6, '안을': 7, '파고드는': 8, '가시가': 9, '되어': 10, '제발': 11, '아주': 12, '아픈': 13, '애써도': 14, '나를': 15, '괴롭히는데': 16, '만큼': 17, '너를': 18, '잊게': 19, '된다면': 20, '차라리': 21, '앓고': 22, '나면': 23, '그만인데': 24, '가시처럼': 25, '깊게': 26, '박힌': 27, '기억은': 28, '아파도': 29, '줄': 30, '모르고': 31}


In [ ]:
text.split("\n\n")

['그대 기억이 지난 사랑이',
 '내 안을 파고드는 가시가 되어',
 '제발 가라고 아주 가라고',
 '애써도 나를 괴롭히는데',
 '아픈 만큼 너를 잊게 된다면',
 '차라리 앓고 나면 그만인데',
 '가시처럼 깊게 박힌 기억은',
 '아파도 아픈 줄 모르고',
 '그대 기억이 지난 사랑이',
 '내 안을 파고드는 가시가 되어',
 '제발 가라고 아주 가라고']

In [ ]:
# 훈련 데이터 만들기

sequence_list = []

for line in text.split("\n\n"): # 줄바꿈을 기준으로 문장나누기
    encoded = tokenizer.texts_to_sequences([line])[0]
    # encoded = line.split(' ')
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequence_list.append(sequence)
sequence_list

In [ ]:
print(sequence_list)

[[2, 3], [2, 3, 4], [2, 3, 4, 5], [6, 7], [6, 7, 8], [6, 7, 8, 9], [6, 7, 8, 9, 10], [11, 1], [11, 1, 12], [11, 1, 12, 1], [14, 15], [14, 15, 16], [13, 17], [13, 17, 18], [13, 17, 18, 19], [13, 17, 18, 19, 20], [21, 22], [21, 22, 23], [21, 22, 23, 24], [25, 26], [25, 26, 27], [25, 26, 27, 28], [29, 13], [29, 13, 30], [29, 13, 30, 31], [2, 3], [2, 3, 4], [2, 3, 4, 5], [6, 7], [6, 7, 8], [6, 7, 8, 9], [6, 7, 8, 9, 10], [11, 1], [11, 1, 12], [11, 1, 12, 1]]


In [ ]:
padded = pad_sequences(sequence_list, padding = 'pre')
padded

In [ ]:
X = padded[:, :-1]
y = padded[:, -1]

In [ ]:
print('original:\n', padded[:3])
print('X\n', X[:3])
print('y\n', y[:3])

original:
 [[0 0 0 2 3]
 [0 0 2 3 4]
 [0 2 3 4 5]]
X
 [[0 0 0 2]
 [0 0 2 3]
 [0 2 3 4]]
y
 [3 4 5]


In [ ]:
X.shape, y.shape

((35, 4), (35,))

In [ ]:
# modeling

model = Sequential([
    tf.keras.layers.Embedding(vocab_size, 50),
    tf.keras.layers.GRU(32, return_sequences = False),
    tf.keras.layers.Dense(vocab_size, activation = 'softmax')
])

In [ ]:
# Functional API

input = tf.keras.Input(shape=(4, ))
x = tf.keras.layers.Embedding(vocab_size, 50)(input)
x = tf.keras.layers.GRU(32, return_sequences = False)(x)
output = tf.keras.layers.Dense(vocab_size, activation = 'softmax')(x)

model = tf.keras.Model(inputs = input, outputs = output)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer ='adam',
              metrics = ['acc'])

In [ ]:
model.fit(X, y, epochs = 300, verbose = 2)

In [ ]:
X[0]

array([0, 0, 0, 2], dtype=int32)

In [ ]:
y_pred = model.predict(X[0:1])

1/1 [==============================] - 0s 37ms/step


In [ ]:
# vocab들 중에 각 단어일 확률
np.argmax(y_pred)

3

In [ ]:
tokenizer.index_word[3]

'기억이'

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=4, padding='pre')

        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence


In [ ]:
print(sentence_generation(model, tokenizer, '제발', 4))

제발 가라고 아주 가라고 아주


In [ ]:
print(sentence_generation(model, tokenizer, '그냥', 5))

그냥 아주 가라고 아주 가라고 아주
